#10장 문서 분류 (Document Classification)

#11-1 나이브 베이즈 분류(Naive Bayes Classifier)

##1.1 직접구현

### Naive Bayes Classifier

In [68]:
training_set = [['me free lottery', 1],
 ['free get free you', 1],
 ['you free scholarship', 0],
 ['free to contact me', 0],
 ['you won award', 0],
 ['you ticket lottery', 1]]

### 토큰 빈도수 및 문서별 토큰수 계산 (확률 계산을 위한 준비)

![대체 텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/98f086c560aa2f66650060277dda4f90e54e30c0)

In [69]:
from collections import defaultdict

# 범주에 속하는 토큰수 세기 1(스팸), 0(정상))
doccnt0 = 0
doccnt1 = 0

# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])

for doc, label in training_set:

    words = doc.split()
    for word in words:
        wordfreq[word][label] += 1

for key, (cnt0, cnt1) in wordfreq.items():
    doccnt0 += cnt0
    doccnt1 += cnt1
    
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'me': [1, 1],
             'free': [2, 3],
             'lottery': [0, 2],
             'get': [0, 1],
             'you': [2, 2],
             'scholarship': [1, 0],
             'to': [1, 0],
             'contact': [1, 0],
             'won': [1, 0],
             'award': [1, 0],
             'ticket': [0, 1]})

In [70]:
doccnt0

10

In [71]:
doccnt1

10

### Training : 토큰별 조건부 확률 계산 

In [72]:
k = 0.5

wordprobs = defaultdict(lambda : [0, 0])
for key, (cnt0, cnt1) in wordfreq.items():
    wordprobs[key][0] = (k + cnt0)/(2*k + doccnt0)
    wordprobs[key][1] = (k + cnt1)/(2*k + doccnt1)
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'me': [0.13636363636363635, 0.13636363636363635],
             'free': [0.22727272727272727, 0.3181818181818182],
             'lottery': [0.045454545454545456, 0.22727272727272727],
             'get': [0.045454545454545456, 0.13636363636363635],
             'you': [0.22727272727272727, 0.22727272727272727],
             'scholarship': [0.13636363636363635, 0.045454545454545456],
             'to': [0.13636363636363635, 0.045454545454545456],
             'contact': [0.13636363636363635, 0.045454545454545456],
             'won': [0.13636363636363635, 0.045454545454545456],
             'award': [0.13636363636363635, 0.045454545454545456],
             'ticket': [0.045454545454545456, 0.13636363636363635]})

### Classify : 신규 텍스트가 주어졌을 때 확률 계산

> 들여쓴 블록



In [73]:
import math

doc = "free lottery"
tokens = doc.split()

# 초기값은 모두 0으로 처리
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복
for word, (prob0, prob1) in wordprobs.items():
    if word in tokens:
        log_prob0 += math.log(prob0)
        log_prob1 += math.log(prob1)

log_prob0 += math.log(doccnt0/(doccnt0 + doccnt1))
log_prob1 += math.log(doccnt1/(doccnt0 + doccnt1))

prob0 = math.exp(log_prob0)
prob1 = math.exp(log_prob1)

print(doc)
print("정상확률 : {:.2f}%".format(prob0 / (prob0 + prob1)*100))
print("스팸확률 : {:.2f}%".format(prob1 / (prob0 + prob1)*100))

free lottery
정상확률 : 12.50%
스팸확률 : 87.50%


In [7]:
log_prob0

-5.265794174842476

In [8]:
prob0

0.00516528925619835

In [9]:
prob1

0.03615702479338842

##1.2 sklearn 활용 (영문 뉴스 분류)

### 뉴스 데이터 다운로드



In [109]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names)
print(twenty_train.data[0])
print(twenty_train.target[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [12]:
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### 문서 분류(파이프 라인 사용)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cvt = CountVectorizer(input = twenty_train.data)
cvt

CountVectorizer(input=["From: lerxst@wam.umd.edu (where's my thing)\n"
                       'Subject: WHAT car is this!?\n'
                       'Nntp-Posting-Host: rac3.wam.umd.edu\n'
                       'Organization: University of Maryland, College Park\n'
                       'Lines: 15\n'
                       '\n'
                       ' I was wondering if anyone out there could enlighten '
                       'me on this car I saw\n'
                       'the other day. It was a 2-door sports car, looked to '
                       'be from the late 60s/\n'
                       'early 70s. It was called a Bri...
                       '"unsafe" econoboxes\n'
                       'instead of Volvos.  I personally take a middle ground '
                       '-- the only\n'
                       'thing I really look for is a three-point seatbelt and '
                       '5+mph bumpers.\n'
                       "I figure that 30mph collisions into brick w

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())]) # 객체에 대한 리스트를 받는다.
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)


In [19]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [110]:
twenty_train.target

array([7, 4, 4, ..., 3, 1, 8])

### Grid Search

In [128]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())]) # 객체에 대한 리스트를 받는다.

parameters_clf = {'vect__ngram_range':[(1,1), (1,2)], 'tfidf__use_idf':(True, False), 'clf__alpha': (1, 0.5, 0.01)}
gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs=-1, verbose=2)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print("Best score : {}".format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimator_.get_params()
print(best_parameters)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best score : 0.9140891823370072
{'memory': None, 'steps': [('vect', CountVectorizer(ngram_range=(1, 2))), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB(alpha=0.01))], 'verbose': False, 'vect': CountVectorizer(ngram_range=(1, 2)), 'tfidf': TfidfTransformer(), 'clf': MultinomialNB(alpha=0.01), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 2), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': None, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True, 'clf__alpha': 0.01, 'clf__class_prior': None, 'clf__fit_prior': T

In [129]:
predicted = gs_clf.best_estimator_.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

NameError: name 'twenty_test' is not defined

vs. https://www.kaggle.com/thomastilli/simple-naive-bayes-classier-0-34974-on-lb

### Parameter 적용

##1.3 sklearn 활용 (한글 뉴스 분류)

In [25]:
!wget https://github.com/kyungsoo-fininsight/mulcam_b/raw/master/data/2019news_1000.csv

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:

%%time
import pandas as pd
df = pd.read_csv("./data/2019news_1000.csv")

Wall time: 671 ms


In [145]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(df['content'], df['category1'], test_size=0.1, random_state=42)

In [146]:
print(train_label.unique())
print(valid_label.unique())

['경제' 'IT/과학' '생활/문화' '정치' '사회']
['정치' '사회' 'IT/과학' '경제' '생활/문화']


In [140]:
from collections import defaultdict
import numpy as np
from konlpy.tag import Mecab

def data_label_preprocess(data, labels, word2id):
    
    preprocessed_label = np.zeros(len(labels),dtype=np.int)
    # preprocessed_label = []
    # print(labels[:10])
    for idx, label in enumerate(labels):
        preprocessed_label[idx] = word2id[label]
        print(preprocessed_label[idx], label, word2id[label])
        # preprocessed_label.append(word2id[label])
        
    
    mecab = Mecab()
    preprocessed_train = []
    for doc in data:
        token_list = []
        for token in mecab.pos(doc):
            if token[1][0] in ['N', 'V']:
                token_list.append(token[0])
        preprocessed_train.append(" ".join(token_list))

    return preprocessed_train, preprocessed_label

In [141]:
word2id = defaultdict(lambda : len(word2id))
for i in train_label.unique():
    word2id[i]
train_data, train_label = data_label_preprocess(train_data, train_label, word2id)
valid_data, valid_label = data_label_preprocess(valid_data, valid_label, word2id)

/문화 2
3 정치 3
3 정치 3
3 정치 3
4 사회 4
4 사회 4
4 사회 4
2 생활/문화 2
3 정치 3
3 정치 3
4 사회 4
2 생활/문화 2
1 IT/과학 1
2 생활/문화 2
2 생활/문화 2
2 생활/문화 2
3 정치 3
0 경제 0
0 경제 0
3 정치 3
1 IT/과학 1
2 생활/문화 2
3 정치 3
3 정치 3
3 정치 3
4 사회 4
1 IT/과학 1
1 IT/과학 1
0 경제 0
4 사회 4
4 사회 4
1 IT/과학 1
1 IT/과학 1
3 정치 3
3 정치 3
4 사회 4
2 생활/문화 2
0 경제 0
4 사회 4
1 IT/과학 1
0 경제 0
2 생활/문화 2
1 IT/과학 1
0 경제 0
4 사회 4
2 생활/문화 2
1 IT/과학 1
0 경제 0
2 생활/문화 2
1 IT/과학 1
3 정치 3
3 정치 3
4 사회 4
2 생활/문화 2
0 경제 0
4 사회 4
4 사회 4
1 IT/과학 1
0 경제 0
4 사회 4
2 생활/문화 2
1 IT/과학 1
2 생활/문화 2
2 생활/문화 2
4 사회 4
4 사회 4
0 경제 0
2 생활/문화 2
2 생활/문화 2
4 사회 4
1 IT/과학 1
2 생활/문화 2
1 IT/과학 1
3 정치 3
1 IT/과학 1
2 생활/문화 2
0 경제 0
0 경제 0
3 정치 3
4 사회 4
3 정치 3
0 경제 0
2 생활/문화 2
3 정치 3
0 경제 0
3 정치 3
4 사회 4
1 IT/과학 1
0 경제 0
3 정치 3
4 사회 4
1 IT/과학 1
4 사회 4
0 경제 0
2 생활/문화 2
0 경제 0
0 경제 0
1 IT/과학 1
4 사회 4
2 생활/문화 2
3 정치 3
1 IT/과학 1
3 정치 3
4 사회 4
0 경제 0
1 IT/과학 1
1 IT/과학 1
3 정치 3
2 생활/문화 2
4 사회 4
1 IT/과학 1
2 생활/문화 2
1 IT/과학 1
3 정치 3
4 사회 4
2 생활/문화 2
4 사회 4
1 IT/과학 1
1 IT/과학 1
2 생활/문화 2
2 생활/문화 2
3

In [117]:
train_label

array([0, 0, 1, ..., 0, 0, 1])

In [142]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())]) # 객체에 대한 리스트를 받는다.

parameters_clf = {'vect__ngram_range':[(1,1), (1,2), (1,3), (1,4)], 'tfidf__use_idf':(True, False), 'clf__alpha': (1, 0.5, 0.01)}
gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs=-1, verbose=0)
gs_clf = gs_clf.fit(train_data, train_label)

print("Best score : {}".format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimator_.get_params()
print(best_parameters)

Best score : 0.7602222222222222
{'memory': None, 'steps': [('vect', CountVectorizer(ngram_range=(1, 3))), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB(alpha=0.01))], 'verbose': False, 'vect': CountVectorizer(ngram_range=(1, 3)), 'tfidf': TfidfTransformer(), 'clf': MultinomialNB(alpha=0.01), 'vect__analyzer': 'word', 'vect__binary': False, 'vect__decode_error': 'strict', 'vect__dtype': <class 'numpy.int64'>, 'vect__encoding': 'utf-8', 'vect__input': 'content', 'vect__lowercase': True, 'vect__max_df': 1.0, 'vect__max_features': None, 'vect__min_df': 1, 'vect__ngram_range': (1, 3), 'vect__preprocessor': None, 'vect__stop_words': None, 'vect__strip_accents': None, 'vect__token_pattern': '(?u)\\b\\w\\w+\\b', 'vect__tokenizer': None, 'vect__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True, 'clf__alpha': 0.01, 'clf__class_prior': None, 'clf__fit_prior': True}


In [143]:
predicted = gs_clf.best_estimator_.predict(valid_data)
np.mean(predicted == valid_label)

0.774

#11-2 서포트 벡터 머신(SVM, Support Vector Machine)

###뉴스 데이터 다운로드

In [147]:
from sklearn.linear_model import SGDClassifier

svm_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))), 
                    ('tfdif', TfidfTransformer()), 
                    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter_no_change=5, random_state=42))])

svm_clf = svm_clf.fit(train_data, train_label)

In [148]:
predicted = svm_clf.predict(valid_data)
np.mean(predicted == valid_label)

0.704